In [1]:
import pandas as pd
from langdetect import detect
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import numpy as np
import sys
import os
import time
from nltk.tag import pos_tag
import urllib.parse
# import enchant
import difflib

In [2]:
# sparql = SPARQLWrapper("http://dbpedia.org/sparql")
# query = """
# PREFIX res: <http://dbpedia.org/resource/>
# SELECT res:?resource 
# FROM <http://dbpedia.org> 
# WHERE 
# { 
# ?resource rdfs:label ?name1 
# FILTER regex(?name1,"Pearl.Harbor", "i") .
# FILTER NOT EXISTS { ?resource rdf:type skos:Concept } . 
# } LIMIT 1"""

# sparql.setQuery(query)
# sparql.setReturnFormat(JSON)

# results = sparql.query().convert()
# # results
# results['results']['bindings'][0]['resource']['value']

In [3]:
# enchant_corrector = enchant.Dict("en_US")
# # enchant_corrector_en = enchant.Dict("en_US")
# # enchant_corrector_de = enchant.Dict("de_DE")
# # enchant_corrector_fr = enchant.Dict("fr_FR")
# # enchant_corrector = {
# #     'en' : enchant_corrector_en,
# #     'de' : enchant_corrector_de,
# #     'fr' : enchant_corrector_fr,
# # }
# def get_predicted_proper_word(my_word):
#     dict,max = {},0
#     a = set(enchant_corrector.suggest(my_word))
#     for b in a:
#        tmp = difflib.SequenceMatcher(None, my_word, b).ratio();
#        dict[tmp] = b
#        if tmp > max:
#           max = tmp
#     return dict[max]

In [4]:
# print(get_predicted_proper_word('helloo'))
# # help(enchant)

In [5]:
# target = pd.read_csv("/home/ngiersz/Projects/zti/data/CEA_Round1_Targets.csv",

target = pd.read_csv(f"D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\semtab2019\\data\\Round 1\\targets\\CEA_Round1_Targets.csv",
                    sep = ',',
                     skiprows=range(112,113), # 112 wadliwy wiersz
                    names=['TableID', 'ColumnID', 'RowID'])
target[:3]

,TableID,ColumnID,RowID
0,50245608_0_871275842592178099,0,154
1,39107734_2_2329160387535788734,1,32
2,22864497_0_8632623712684511496,0,227


In [18]:
# string = "he$eja k> {.} f"
# removeSpecialChars = string.translate({ord(c): ".{0,1}" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
# removeSpecialChars

'he.{0,1}eja k.{0,1} .{0,1}.{0,1}.{0,1} f'

In [13]:
string = "he$eja k> {.} f"
''.join(e for e in string if e.isalnum() or e.isspace())

'heeja k  f'

In [35]:
cell_content = "Pearl Harbor"
film_year = str(2001)
query = """PREFIX res: <http://dbpedia.org/resource/>
                    SELECT res:?resource 
                    FROM <http://dbpedia.org> 
                    WHERE 
                    {
                        {
                            ?resource rdf:type dbo:Film .
                            ?resource rdfs:label ?name1 FILTER regex(?name1, "^%s$" , "i")
                        }
                        UNION
                        {
                            ?resource rdf:type dbo:Film .
                            ?resource rdfs:label ?name1 FILTER regex(?name1, "^The %s$", "i")
                        }
                        UNION
                        {
                            ?resource rdf:type dbo:Film .
                            ?resource rdfs:label "%s"@en .
                            ?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, "%s")
                        }
                        UNION
                        {
                            ?resource rdf:type dbo:Film .
                            ?resource rdfs:label "%s"@en .
                            ?resource rdfs:label ?name2 FILTER regex(?name2, "%s")
                        }
                        UNION
                        {
                            ?resource rdf:type dbo:Film .
                            ?resource rdfs:label ?name1 FILTER regex(?name1, "%s", "i") .
                            ?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, "%s")
                        }
                        UNION
                        {
                            ?resource rdf:type dbo:Film .
                            ?resource rdfs:label ?name1 FILTER regex(?name1, "^%s", "i") .
                            ?resource rdfs:label ?name2 FILTER regex(?name2, "%s")
                        }
                        UNION
                        {
                            ?resource rdf:type dbo:Film .
                            ?resource rdfs:label ?name1 FILTER regex(?name1, "^%s", "i")
                        }
                    } LIMIT 1""" % (cell_content, cell_content, cell_content, film_year, cell_content, film_year, cell_content, film_year, cell_content, film_year, cell_content)

query = query.replace("\n", ' ')



'PREFIX res: <http://dbpedia.org/resource/>                     SELECT res:?resource                      FROM <http://dbpedia.org>                      WHERE                      {                         {                             ?resource rdf:type dbo:Film .                             ?resource rdfs:label ?name1 FILTER regex(?name1, "^Pearl Harbor$" , "i")                         }                         UNION                         {                             ?resource rdf:type dbo:Film .                             ?resource rdfs:label ?name1 FILTER regex(?name1, "^The Pearl Harbor$", "i")                         }                         UNION                         {                             ?resource rdf:type dbo:Film .                             ?resource rdfs:label "Pearl Harbor"@en .                             ?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, "2001")                         }                         UNION                       

In [40]:
def replace_characters(chars_to_replace_list_of_dicts, cell_contents_list, cell_content):
    # zamienia wszystkie znaki odpowiadające kluczowi np. '[?]' na to co w wartości ''
    # każdy znak zamienia osobno. Nie wyszukuje stringa '[?]', tylko każdy znak po kolei szukam i zamieniam
    for dicts in chars_to_replace_list_of_dicts:
        for old_chars, new_char in dicts.items():
            for old_char in old_chars:
#                 print(f'Before replace: {cell_content}')
                cell_content = cell_content.replace(old_char, new_char)
#                 print(f'After replace: {cell_content}')                
            if cell_content not in cell_contents_list:
                cell_contents_list.append(cell_content)
    return cell_contents_list
#     print(f'Before replace: {cell_content}')
#     cell_content = cell_content.replace('[?]', '')
#     cell_content = cell_content.replace(',', '')
#     cell_content = cell_content.replace('-', '')
#     print(f'After replace: {cell_content}')                
#     if cell_content not in cell_contents_list:
#         cell_contents_list.append(cell_content)
#     return cell_contents_list

# def spellcheck_words(cell_contents_list, cell_content):
#     print(f"SPELL CHECKING {cell_content}")
#     words = cell_content.split()
#     for i, word in enumerate(words):
#         if '?' in word:
#             print(word)
#             if not enchant_corrector.check(word):
#                 new_word = get_predicted_proper_word(word)
#                 if new_word != word:
#                     print(f'Before spellcheck correction: {cell_content}')
#                     cell_content = cell_content.replace(word, new_word)
#                     print(f'After spellcheck correction: {cell_content}')
                    
#     if cell_content not in cell_contents_list:
#         cell_contents_list.append(cell_content)
#     return cell_contents_list

def create_queries(cell_content):
    begin_of_query = """PREFIX res: <http://dbpedia.org/resource/>
                        SELECT res:?resource 
                        FROM <http://dbpedia.org> """
    end_of_query = 'LIMIT 1'
    
    cell_content_copy = cell_content
    cell_contents_list = []
    cell_contents_list.append(cell_content) # oryginalne
    cell_contents_list.append(str('The ' + cell_content)) # opcja z The na początku
    cell_contents_list.append(cell_content.capitalize())
    cell_contents_list.append(cell_content.title())
    
    # pojawiały się wartości typy "Guardian The", a powinno być "The Gurdian", 
    # albo "Thunderbird, Lake", a powinno być "Lake Thunderbird"
    if re.search(", [a-zA-Z]*$", cell_content_copy):
        found_string = re.search(", [a-zA-Z]*$", cell_content_copy).group(0)
        new_cell_content = cell_content_copy.replace(found_string, '')
        found_string = found_string.replace(', ','')
        new_cell_content = found_string.capitalize() + ' ' + new_cell_content
        
        cell_contents_list.append(new_cell_content)
    
    # pojawiały się wartości typy "Guardian The", a powinno być "The Gurdian" to samo z A i An
    if re.search(" The$", cell_content):
        index = cell_content.find(" The")
        cell_contents_list.append("The " + cell_content[:index].replace(',','').strip())
        cell_contents_list.append(cell_content[:index].strip())
    elif re.search(" A$", cell_content):
        index = cell_content.find(" A")
        cell_contents_list.append("A " + cell_content[:index].replace(',','').strip())
        cell_contents_list.append(cell_content[:index].strip())
    elif re.search(" An$", cell_content):
        index = cell_content.find(" An")
        cell_contents_list.append("An " + cell_content[:index].replace(',','').strip())
        cell_contents_list.append(cell_content[:index].strip())

    # usunięcię The z początku
    if re.search("^The ", cell_content):
        cell_contents_list.append(re.sub('^The ', '', cell_content))


#     strings_to_replace_dict = [{'[?]':''}, {',':''}, {'-':' '}]
    strings_to_replace_dict = [{'[?]':''}, {',':''}, {'-':' '}, {'?':''}] # dict = {chars_to_replace:new_char}
    cell_contents_list = replace_characters(strings_to_replace_dict, cell_contents_list, cell_content)
    
#     if '?' in cell_content:
#         cell_contents_list = spellcheck_words(cell_contents_list, cell_content) 
        
    list_of_queries = []
    # pierwsze zapytanie - bierze zawartość sformatowanej komórki i wyszukuje linku,
    query = begin_of_query + 'WHERE { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + ' FILTER NOT EXISTS { ?resource rdf:type skos:Concept } .}' + end_of_query
    list_of_queries.append(query)
    

    for cell_content in cell_contents_list:
        query = begin_of_query + 'WHERE { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + ' FILTER NOT EXISTS { ?resource rdf:type skos:Concept } .} ' + end_of_query
        list_of_queries.append(query)


    lang = 'en'
    try:
        lang = detect(cell_content)
    except:
        pass

    if lang != 'en':
        for cell_content  in cell_contents_list:
            query = begin_of_query + 'WHERE { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + '@' + str(lang) + ' FILTER NOT EXISTS { ?resource rdf:type skos:Concept } .} ' + end_of_query
            list_of_queries.append(query)
            
#     strings_to_replace_dict = [{"~`!@#$%^&*()-_=+:;{}[]\"<>?,./|\'":'.{0,1}'}]
#     cell_content_regex = replace_characters(strings_to_replace_dict, [], cell_content)[0]

#     cell_content_regex = cell_content.translate({ord(c): ".{0,1}" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
#     query = begin_of_query + 'WHERE { ?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^' + str(cell_content_regex) + '$", "i")  FILTER NOT EXISTS { ?resource rdf:type skos:Concept } }' + end_of_query
#     list_of_queries.append(query)
    
    return list_of_queries

def create_query_for_films(cell_content):
    begin_of_query = """PREFIX res: <http://dbpedia.org/resource/>
                        SELECT res:?resource 
                        FROM <http://dbpedia.org> """
    end_of_query = 'LIMIT 1'
    
    row = df[row_id-1:row_id]
    film_year = row['Year'].values[0]

    cell_content = cell_content.replace(',','')
    cell_content = cell_content.replace('The','')
    cell_content = cell_content.replace(',','')
    cell_content = cell_content.strip()

    query = """PREFIX res: <http://dbpedia.org/resource/>
                SELECT res:?resource 
                FROM <http://dbpedia.org> 
                WHERE 
                {
                    {
                        ?resource rdf:type dbo:Film .
                        ?resource rdfs:label ?name1 FILTER regex(?name1, "^%s$" , "i")
                    }
                    UNION
                    {
                        ?resource rdf:type dbo:Film .
                        ?resource rdfs:label ?name1 FILTER regex(?name1, "^The %s$", "i")
                    }
                    UNION
                    {
                        ?resource rdf:type dbo:Film .
                        ?resource rdfs:label "%s"@en .
                        ?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, "%s")
                    }
                    UNION
                    {
                        ?resource rdf:type dbo:Film .
                        ?resource rdfs:label "%s"@en .
                        ?resource rdfs:label ?name2 FILTER regex(?name2, "%s")
                    }
                    UNION
                    {
                        ?resource rdf:type dbo:Film .
                        ?resource rdfs:label ?name1 FILTER regex(?name1, "%s", "i") .
                        ?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, "%s")
                    }
                    UNION
                    {
                        ?resource rdf:type dbo:Film .
                        ?resource rdfs:label ?name1 FILTER regex(?name1, "^%s", "i") .
                        ?resource rdfs:label ?name2 FILTER regex(?name2, "%s")
                    }
                    UNION
                    {
                        ?resource rdf:type dbo:Film .
                        ?resource rdfs:label ?name1 FILTER regex(?name1, "^%s", "i")
                    }
                } LIMIT 1""" % (cell_content, cell_content, cell_content, film_year, cell_content, film_year, cell_content, film_year, cell_content, film_year, cell_content)

    query = query.replace("\n", ' ')

#         list_of_queries = [query1, query2, query3, query4, query5, query6, query7]

    return query


def send_query(query):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    # jeśli wystąpi błąd z połączeniem się z serwerem to próbujemy aż do skutku
    results = None
    while results is None:
        try:
            results = sparql.query().convert()
        except Exception as e:
            print("EXCEPTION:" + str(e))
            print("TAKI: " + str(cell_content))
            continue
            
    return results

def get_URI_from_results(results, query, verbose=0):
    # jeśli nic nie znaleziono no to trudno, nie mamy linku, wstawiamy NOT FOUND URI

    if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
        # printy przydatne do sprawdzania jakich zawartości nam nie wyszukał,
        # trzeba analizowac i dodawać kolejne warunki, zmieniać cell_content na różne sposoby
        if verbose:
            print(table_id, column_id, row_id)
            print(cell_content_copy)
            print(query)    
        return 'NOT FOUND URI'


    # jeśli jednak mamy jakies linki to wyciągamy go z jsona,
    # linków może być więcej niż 1, aktualnie bierzemy pierwszy lepszy
    uri = results['results']['bindings'][0]['resource']['value']
    return uri
#     results_list = results['results']['bindings']
#     for result_dict in results_list:
#         uri = result_dict['resource']['value']
#         match = re.match("http://dbpedia.org/resource/*", uri)

#         # czasami pojawiają dziwne linki np. http://dbpedia.org/resource/Category:Max_Weber
#         # my takich nie chcemy i musimy ich unikać
#         # interesują nas tylko np.  http://dbpedia.org/resource/Max_Weber
#         # więc to jest warunek pozbywajacy się wszystkich linków z Category:
#         if match and uri[5:].find("Category:") == -1:
# #                 answers_dict['URI'].append(str(uri)
# #                 formatted_uri = 'http://dbpedia.org/resource/' + str(urllib.parse.quote(uri[28:]))
# #                 formatted_uri = formatted_uri.replace('-','%E2%80%93')
#             return uri
    
    return 'NOT FOUND URI'

In [ ]:
start = time.time()

# słownik, z którego będzie tworzony wynikowy df
answers_dict = {"TableID":[],
               "ColumnID":[],
               "RowID":[],
               "URI":[]}

# zmienna, aby ograniczyć liczbę przetwarzanych wierszy, niżej masz 3 linijki skomentowane,
# odkomentowujesz je i ustawiasz sobię liczbę wierszy w tym warunku, aktualnie 1000
counter = 0
# zlicza ile linków nie zostało znalezionych w ogóle,
# tzn. po wysłaniu zapytania nie dostaliśmy żądnej odpwiedzi od sparql (wiersze oznaczone przez NOT FOUND URI)
not_found_uris = 0

for index, row in target.iterrows():
    try:
        counter = counter + 1
        if counter == 200:
            break


        table_id = row['TableID']
        column_id = row['ColumnID']
        row_id = row['RowID']
        
        # wczytanie odpowiedniej tabeli
#         df = pd.read_csv(f"/home/ngiersz/Projects/zti/data/CEA_Round1/{table_id}.csv",
#                     sep=',')
        
        df = pd.read_csv(f"D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\semtab2019\\data\\Round 1\\tables\\{table_id}.csv",
            sep=',')
        
            
        # wyciągniecie wartości komórki, która nas interesuje
        cell_content = df.iloc[row_id-1:row_id, column_id:(column_id+1)].values[0][0]
        cell_content_copy = cell_content
        
        print(str(counter) + ' ' + cell_content)
        
        # usunięcie białych znaków z początku i końca stringa
        cell_content = cell_content.strip()
        
        list_of_queries = []
        if len({'Title', 'Year'}.intersection(set(df.columns))) == 2:
            list_of_queries.append(create_query_for_films(cell_content))
        list_of_queries = list_of_queries + create_queries(cell_content)
        
        for id, query in enumerate(list_of_queries):
            results = send_query(query)
            uri = get_URI_from_results(results, query, verbose=0)
            if uri != 'NOT FOUND URI':             
                answers_dict['TableID'].append(table_id)
                answers_dict['ColumnID'].append(column_id)
                answers_dict['RowID'].append(row_id)
                answers_dict['URI'].append(uri)
                break
            elif id+1 == len(list_of_queries) and uri == 'NOT FOUND URI':
                answers_dict['TableID'].append(table_id)
                answers_dict['ColumnID'].append(column_id)
                answers_dict['RowID'].append(row_id)
                answers_dict['URI'].append("NOT FOUND URI")
                not_found_uris += 1
                
                print(table_id, column_id, row_id)
                print(cell_content_copy)
                
                    

        # aktualnie nie używane, ale pewnie będzie
        # pos_tag() - kategoryzuje słowa, np. znajduje czasowniki, rzeczowniki, nazwy własne (jak imiona i nazwiska)
        # chodzi o to, że czasami odpowiedzi są w angielskiej dbpedii,
        # np. Alex Rodriguez wykrywa jako język ca - pewnie kataloński, 
        # ale nie ma jego odpowiednika w katalońskiej dbpedii, a w angielskiej jest
        # dlatego chcę wykrywać nazwy wkłasne jak imiona i nazwiska i wtedy zawsze szukać w angielksiej dbpedii,
        # ale z tym też był problem czasami i na razie nie myślałem nad tym dalej
        # NNP - to kategoria nazwy własnej, np. imię
#         tags = pos_tag(cell_content.split())
#         is_one_NNP = False
#         for tag in tags:
#             if tag[1] == "NNP":
#                 is_one_NNP = True
#                 break
#         try:
#             if is_one_NNP:
#                 lang = '@en'
#             else:
#                 lang = str('@') + str(detect(cell_content))
#         except Exception:
#             lang = ''

#         list_of_queries = create_queries(cell_content)
#         for query in list_of_queries:
#             results = send_query(query)                              
#             uri = get_URI_from_results(results, query, verbose=1) 
                         
#             answers_dict['TableID'].append(table_id)
#             answers_dict['ColumnID'].append(column_id)
#             answers_dict['RowID'].append(row_id)
#             answers_dict['URI'].append(uri)
#             if uri == 'NOT FOUND URI':
#                 not_found_uris += 1
#                 continue

    except Exception as e:
        # na początku były problemy z danymi, naprawiłem
        # potem zaczęły pojawiać sę problemy, żę serwer czasami nie odpowiadał,
        # więc jak jest error to olewamy i przechodzimy do następnego wiersza
        # będzie trzeba zrobić, że jak serwer nie odpowie z jakiegoś powodu to trzeba ponowić to samo zapytanie
        # zamiast pomijać, bo tracimy wiersze
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(table_id, column_id, row_id) # jaki plik, kolumna i wiersz
        print(cell_content_copy) # zawartość komórki
        print(e) # jaki błąd, teraz chyba tylko z brakiem połączenia z serwerem
        print(exc_tb.tb_lineno) # w jakiej liniii błąd
        continue



# print(answers_dict)
result_df = pd.DataFrame(answers_dict)
end = time.time()
print(end-start)
print(not_found_uris)

1 Pearl Harbor
2 Minneapolis-Saint Paul International Airport
3 Need for Speed: Carbon
4 Alex Rodriguez
5 King Kong
6 School of Rock , The
7 Lex Luger
8 Crash Bash
9 The Hurt Locker
10 Lion
11 Lac-Seul
12 Fear and Loathing in Las Vegas
13 Max Weber
14 Kidsty Pike
15 Randy Johnson
16 TimeShift
17 Batman: Arkham City
18 Dazzy Vance?
19 A Clockwork Orange
20 Bill Collins
21 Greenland
22 Thunbergia grandiflora
23 Peter of Sebaste[?]
24 Jimmy Ryan
25 State Hermitage Museum
26 Mario Party 8
27 Lygodium palmatum
28 New Taiwan Dollar
29 Mary Poppins
30 The Curse of Monkey Island
31 Super Mario All-Stars + Super Mario World
32 Lappet-faced vulture
33 Spiderman
34 Grey-backed fiscal
35 Heat
36 Pope Linus
37 Broken Bow Lake
38 TREE SWALLOW
39 The Wild Bunch
40 FIFA 08
41 The Legend of Zelda: Ocarina of Time
42 Fellbarrow
43 Sim City
44 Asparagus densiflorus
45 To Be or Not to Be
46 John of the Cross
47 Mark Langston
48 Soldier of Fortune: Pay Back
49 One Hour Photo
50 Brown parrot
51 My Cousin Vi

In [ ]:
print(result_df.shape)
print(target.shape)
result_df[:20]
# result_df[10:13]

In [ ]:
# zapisanie submission
# przez evaluator codes można sprawdzić wyniki
result_df.to_csv("/home/ngiersz/Projects/zti/ZTI_projekt/evaluator_codes/evaluator_codes/data/my_submission_pyenchant.csv",
                 index=False,
                 sep=',',
                header=False)

### NIEISTOTNE, TESTY ITP.

In [ ]:
df = pd.read_csv("D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\CEA_Round1\\13719111_1_5719401842463579519.csv",
                sep=',')
df[:3]

In [ ]:
detect(df.iloc[0:1, 1:2].values[0][0])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Norway> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
for result in results["results"]["bindings"]:
    print(result["label"]["value"])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ont: <http://dbpedia.org/ontology/>
    SELECT ?a, ?c
    WHERE { 
            ?a ont:location <http://dbpedia.org/resource/Delft> . 
            FILTER regex(?a, "Ikea", "i")
          }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
# print(result['a'])
# print(result['b'])
# print(result['c'])

# for result in results["results"]["bindings"]:
#     print(type(result))

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {e}
    VALUES (?r) {e} ({source}) {w}
        {e} ?r ?p ?o {w}
        UNION
        {e} ?s ?r ?o {w}
        UNION
        {e} ?s ?p ?r {w}
    {w} 
'''.format(source=sss, e="{", w="}"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {
    VALUES (?r) { (dbr:Gra_o_tron) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
    } 
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)



In [ ]:
SELECT ?r ?p ?o
WHERE {
 VALUES (?r) { (dbr:Cat) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
}

select ?s
where {
        { ?s rdfs:label "Kot domowy"@pl }
        UNION
        { "Kot domowy"@pl rdfs:label ?s }
} 

SELECT ?r
WHERE {
        { ?r rdfs:label "Kaspisches Meer"@de }

}

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?resource
    WHERE { ?resource rdfs:label "Kaspisches Meer"@de }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# print(results['results']['bindings'][1]['r']['value'])
# results = str(results['results']['bindings'])
print(str(results['results']['bindings']))
if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
    print("nie ma")
else:
    print('jest')
# for result_dict in results_list:
#     uri = result_dict['resource']['value']
#     match = re.match("http://dbpedia.org/resource/*", uri)
#     if match:
#         print(uri)
#     else:
#         print("NIE")

# print(results['results']['bindings'])

In [ ]:
import re
match = re.match("http://dbpedia.org/resource/*",'http://dbpedia.org/resource/Caspian_Sea', re.M|re.I)
if match:
    print(match.group())